In [1]:
import os
import pandas as pd

# DataFrame 초기화
df = pd.DataFrame(columns=['user_ID', 'trial_ID', 'task_ID', 'gx', 'gy', 'gz', 'label'])

# task_id와 매핑되는 label 정보 생성
label_mapping = {
    'D01': 'walking',
    'D03': 'jogging',
    'D05': 'stairs_walking',
    'D07': 'standing',
    'F06': 'fall'
}

# 데이터 폴더 경로
data_folder = "../data/SisFall_dataset"

# 폴더 내의 파일들을 확인하고 처리
record_count = 0  # 전체 파일에서의 레코드 개수
for root, dirs, files in os.walk(data_folder):
    for dir_name in dirs:
        dir_path = os.path.join(root, dir_name)
        user_ID = dir_name

        for file_name in os.listdir(dir_path):
            if file_name.endswith(".txt"):
                # 파일 경로
                file_path = os.path.join(dir_path, file_name)

                # 파일 이름에 대한 정보
                activity, _, trial_ID_with_extension = file_name.split('_')
                trial_ID = trial_ID_with_extension[:-4]  # .txt 제거

                # task_id가 D01, D03, D05, D07, F06인 경우에만 처리
                if activity in ['D01', 'D03', 'D05', 'D07', 'F06']:
                    # task_id에 해당하는 label 정보
                    task_ID = activity
                    label = label_mapping[task_ID]

                    # Angular velocity 변환을 위한 공식에 사용되는 값들
                    Range = 2000  # 예시 값을 사용
                    Resolution = 16  # 예시 값을 사용

                    # 파일에서 데이터를 읽어와서 처리
                    with open(file_path, 'r') as file:
                        lines = file.readlines()

                    # 데이터 파싱 및 계산
                    gx_values = []
                    gy_values = []
                    gz_values = []

                    for line in lines:
                        # 쉼표를 기준으로 줄을 분리하여 필드의 값을 가져오기
                        fields = line.strip().split(',')

                        # gx, gy, gz 값 얻기 및 변환 계산
                        gx_value = (2 * Range / (2 ** Resolution)) * float(fields[3])
                        gy_value = (2 * Range / (2 ** Resolution)) * float(fields[4])
                        gz_value = (2 * Range / (2 ** Resolution)) * float(fields[5])

                        gx_values.append(gx_value)
                        gy_values.append(gy_value)
                        gz_values.append(gz_value)

                        if len(gx_values) == 1:
                            # 한 줄의 레코드 처리가 완료되었으므로 DataFrame에 추가
                            df.loc[len(df)] = {
                                'user_ID': user_ID,
                                'trial_ID': trial_ID,
                                'task_ID': task_ID,
                                'gx': gx_values[0],
                                'gy': gy_values[0],
                                'gz': gz_values[0],
                                'label': label
                            }
                            record_count += 1
                            gx_values = []
                            gy_values = []
                            gz_values = []

                            if record_count == 40:
                                # 40개의 레코드 처리가 완료되었으므로 다음 파일 처리
                                record_count = 0
                                break

                if record_count == 40:
                    # 40개의 레코드 처리가 완료되었으므로 다음 파일 처리
                    break

        if record_count == 40:
            # 40개의 레코드 처리가 완료되었으므로 다음 파일 처리
            break

    if record_count == 40:
        # 40개의 레코드 처리가 완료되었으므로 다음 파일 처리
        break

print(df)

      user_ID trial_ID task_ID        gx         gy         gz     label
0        SA01      R01     D01 -1.098633 -30.761719 -21.484375   walking
1        SA01      R01     D01 -3.234863 -34.667969 -18.676758   walking
2        SA01      R01     D01 -5.126953 -37.414551 -16.540527   walking
3        SA01      R01     D01 -6.347656 -39.489746 -13.854980   walking
4        SA01      R01     D01 -7.812500 -41.198730 -11.657715   walking
...       ...      ...     ...       ...        ...        ...       ...
23035    SE15      R05     D07 -2.380371   5.981445   0.305176  standing
23036    SE15      R05     D07 -2.197266   6.408691   0.366211  standing
23037    SE15      R05     D07 -1.892090   6.835938   0.427246  standing
23038    SE15      R05     D07 -1.647949   7.263184   0.427246  standing
23039    SE15      R05     D07 -1.525879   7.568359   0.427246  standing

[23040 rows x 7 columns]


In [2]:
# CSV 파일로 저장
csv_path = "../data/SisFall_df.csv"
df.to_csv(csv_path, index=False)
print("CSV 파일이 저장되었습니다:", csv_path)

CSV 파일이 저장되었습니다: ../data/SisFall_df.csv


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Shuffle the DataFrame
df_shuffled = df.sample(frac=1, random_state=42)

# Split data into input features (X) and target variable (y)
X = df_shuffled[['gx', 'gy', 'gz']].values
y = df_shuffled['label'].values

# Encode labels using LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Convert encoded labels to one-hot encoding
y_one_hot = to_categorical(y_encoded)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Define the LSTM model
model = Sequential()
model.add(LSTM(64, input_shape=(3, 1), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [5]:
# Reshape input features to match the expected shape of the LSTM model
X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_reshaped = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Train the LSTM model
model.fit(X_train_reshaped, y_train, epochs=100, batch_size=32)

# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test_reshaped, y_test)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

Epoch 1/10
576/576 [==============================] - 4s 2ms/step - loss: 1.3098 - accuracy: 0.4277
Epoch 2/10
576/576 [==============================] - 1s 2ms/step - loss: 1.2512 - accuracy: 0.4579
Epoch 3/10
576/576 [==============================] - 1s 2ms/step - loss: 1.2297 - accuracy: 0.4692
Epoch 4/10
576/576 [==============================] - 1s 2ms/step - loss: 1.2099 - accuracy: 0.4870
Epoch 5/10
576/576 [==============================] - 1s 2ms/step - loss: 1.1905 - accuracy: 0.4980
Epoch 6/10
576/576 [==============================] - 1s 2ms/step - loss: 1.1740 - accuracy: 0.5023
Epoch 7/10
576/576 [==============================] - 1s 2ms/step - loss: 1.1611 - accuracy: 0.5092
Epoch 8/10
576/576 [==============================] - 1s 2ms/step - loss: 1.1487 - accuracy: 0.5104
Epoch 9/10
576/576 [==============================] - 1s 2ms/step - loss: 1.1387 - accuracy: 0.5157
Epoch 10/10
144/144 [==============================] - 1s 1ms/step - loss: 1.0814 - accuracy: 0.5384

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical

# DataFrame을 무작위로 섞기
df_shuffled = df.sample(frac=1, random_state=42)

# 데이터를 입력 특성 (X)과 타겟 변수 (y)로 분할하기
X = df_shuffled[['gx', 'gy', 'gz']].values
y = df_shuffled['label'].values

# 레이블을 인코딩하기 위해 LabelEncoder 사용
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# 인코딩된 레이블을 원-핫 인코딩으로 변환하기
y_one_hot = to_categorical(y_encoded)

# 데이터를 훈련 세트와 테스트 세트로 분할하기
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

# 입력 특성을 LSTM 모델의 예상 형태로 재구성
X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_reshaped = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# LSTM 모델 정의
model = Sequential()
model.add(LSTM(64, input_shape=(3, 1), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# LSTM 모델 훈련
model.fit(X_train_reshaped, y_train, epochs=100, batch_size=32)

# 훈련 데이터에서 모델 평가
train_loss, train_accuracy = model.evaluate(X_train_reshaped, y_train)
print(f'Train Loss: {train_loss:.4f}')
print(f'Train Accuracy: {train_accuracy:.4f}')

# 테스트 데이터에서 모델 평가
test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


Epoch 1/100
576/576 [==============================] - 4s 2ms/step - loss: 1.3078 - accuracy: 0.4255
Epoch 2/100
576/576 [==============================] - 1s 2ms/step - loss: 1.2490 - accuracy: 0.4590
Epoch 3/100
576/576 [==============================] - 1s 2ms/step - loss: 1.2243 - accuracy: 0.4773
Epoch 4/100
576/576 [==============================] - 1s 2ms/step - loss: 1.2021 - accuracy: 0.4868
Epoch 5/100
576/576 [==============================] - 1s 2ms/step - loss: 1.1889 - accuracy: 0.4962
Epoch 6/100
576/576 [==============================] - 1s 2ms/step - loss: 1.1726 - accuracy: 0.5046
Epoch 7/100
576/576 [==============================] - 1s 2ms/step - loss: 1.1559 - accuracy: 0.5133
Epoch 8/100
576/576 [==============================] - 1s 2ms/step - loss: 1.1451 - accuracy: 0.5188
Epoch 9/100
576/576 [==============================] - 1s 2ms/step - loss: 1.1311 - accuracy: 0.5258
Epoch 10/100
576/576 [==============================] - 1s 2ms/step - loss: 1.1257 - accura

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical

# DataFrame을 무작위로 섞기
df_shuffled = df.sample(frac=1, random_state=42)

# 데이터를 입력 특성 (X)과 타겟 변수 (y)로 분할하기
X = df_shuffled[['gx', 'gy', 'gz']].values
y = df_shuffled['label'].values

# 레이블을 인코딩하기 위해 LabelEncoder 사용
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# 인코딩된 레이블을 원-핫 인코딩으로 변환하기
y_one_hot = to_categorical(y_encoded)

# 데이터를 훈련 세트와 테스트 세트로 분할하기
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

# 입력 특성을 LSTM 모델의 예상 형태로 재구성
X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_reshaped = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# LSTM 모델 정의
model = Sequential()
model.add(LSTM(64, input_shape=(3, 1), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# LSTM 모델 훈련
model.fit(X_train_reshaped, y_train, epochs=1000, batch_size=32)

# 훈련 데이터에서 모델 평가
train_loss, train_accuracy = model.evaluate(X_train_reshaped, y_train)
print(f'Train Loss: {train_loss:.4f}')
print(f'Train Accuracy: {train_accuracy:.4f}')

# 테스트 데이터에서 모델 평가
test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

Epoch 1/1000
576/576 [==============================] - 4s 2ms/step - loss: 1.3131 - accuracy: 0.4223
Epoch 2/1000
576/576 [==============================] - 1s 2ms/step - loss: 1.2505 - accuracy: 0.4562
Epoch 3/1000
576/576 [==============================] - 1s 2ms/step - loss: 1.2280 - accuracy: 0.4720
Epoch 4/1000
576/576 [==============================] - 1s 2ms/step - loss: 1.2093 - accuracy: 0.4856
Epoch 5/1000
576/576 [==============================] - 1s 2ms/step - loss: 1.1935 - accuracy: 0.4913
Epoch 6/1000
576/576 [==============================] - 1s 2ms/step - loss: 1.1761 - accuracy: 0.4988
Epoch 7/1000
576/576 [==============================] - 1s 2ms/step - loss: 1.1578 - accuracy: 0.5086
Epoch 8/1000
576/576 [==============================] - 1s 2ms/step - loss: 1.1481 - accuracy: 0.5124
Epoch 9/1000
576/576 [==============================] - 1s 2ms/step - loss: 1.1369 - accuracy: 0.5184
Epoch 10/1000
576/576 [==============================] - 1s 2ms/step - loss: 1.129

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical

# DataFrame을 무작위로 섞기
df_shuffled = df.sample(frac=1, random_state=42)

# 데이터를 입력 특성 (X)과 타겟 변수 (y)로 분할하기
X = df_shuffled[['gx', 'gy', 'gz']].values
y = df_shuffled['label'].values

# 레이블을 인코딩하기 위해 LabelEncoder 사용
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# 인코딩된 레이블을 원-핫 인코딩으로 변환하기
y_one_hot = to_categorical(y_encoded)

# 데이터를 훈련 세트와 테스트 세트로 분할하기
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

# 입력 특성을 LSTM 모델의 예상 형태로 재구성
X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_reshaped = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# LSTM 모델 정의
model = Sequential()
model.add(LSTM(64, input_shape=(3, 1), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# LSTM 모델 훈련
model.fit(X_train_reshaped, y_train, epochs=3000, batch_size=32)

# 훈련 데이터에서 모델 평가
train_loss, train_accuracy = model.evaluate(X_train_reshaped, y_train)
print(f'Train Loss: {train_loss:.4f}')
print(f'Train Accuracy: {train_accuracy:.4f}')

# 테스트 데이터에서 모델 평가
test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical

# DataFrame을 무작위로 섞기
df_shuffled = df.sample(frac=1, random_state=42)

# 데이터를 입력 특성 (X)과 타겟 변수 (y)로 분할하기
X = df_shuffled[['gx', 'gy', 'gz']].values
y = df_shuffled['label'].values

# 레이블을 인코딩하기 위해 LabelEncoder 사용
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# 인코딩된 레이블을 원-핫 인코딩으로 변환하기
y_one_hot = to_categorical(y_encoded)

# 데이터를 훈련 세트와 테스트 세트로 분할하기
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

# 입력 특성을 LSTM 모델의 예상 형태로 재구성
X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_reshaped = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# LSTM 모델 정의
model = Sequential()
model.add(LSTM(64, input_shape=(3, 1), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# LSTM 모델 훈련
model.fit(X_train_reshaped, y_train, epochs=3000, batch_size=64)

# 훈련 데이터에서 모델 평가
train_loss, train_accuracy = model.evaluate(X_train_reshaped, y_train)
print(f'Train Loss: {train_loss:.4f}')
print(f'Train Accuracy: {train_accuracy:.4f}')

# 테스트 데이터에서 모델 평가
test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

Epoch 1/3000
288/288 [==============================] - 3s 3ms/step - loss: 1.3403 - accuracy: 0.4214
Epoch 2/3000
288/288 [==============================] - 1s 3ms/step - loss: 1.2622 - accuracy: 0.4517
Epoch 3/3000
288/288 [==============================] - 1s 3ms/step - loss: 1.2399 - accuracy: 0.4628
Epoch 4/3000
288/288 [==============================] - 1s 3ms/step - loss: 1.2204 - accuracy: 0.4755
Epoch 5/3000
288/288 [==============================] - 1s 3ms/step - loss: 1.2061 - accuracy: 0.4877
Epoch 6/3000
288/288 [==============================] - 1s 3ms/step - loss: 1.1927 - accuracy: 0.4906
Epoch 7/3000
288/288 [==============================] - 1s 3ms/step - loss: 1.1784 - accuracy: 0.5034
Epoch 8/3000
288/288 [==============================] - 1s 3ms/step - loss: 1.1698 - accuracy: 0.5055
Epoch 9/3000
288/288 [==============================] - 1s 3ms/step - loss: 1.1567 - accuracy: 0.5148
Epoch 10/3000
288/288 [==============================] - 1s 3ms/step - loss: 1.150

In [12]:
from sklearn.preprocessing import MinMaxScaler

# MinMaxScaler 인스턴스 생성
scaler = MinMaxScaler(feature_range=(-1, 1))

# 'gx', 'gy', 'gz' 열의 데이터 추출
data = df[['gx', 'gy', 'gz']].values

# 데이터 스케일링
scaled_data = scaler.fit_transform(data)

# 스케일링된 값을 새로운 열에 저장
df['scaled_gx'] = scaled_data[:, 0]
df['scaled_gy'] = scaled_data[:, 1]
df['scaled_gz'] = scaled_data[:, 2]

print(df.head())

  user_ID trial_ID task_ID        gx         gy         gz    label   
0    SA01      R01     D01 -1.098633 -30.761719 -21.484375  walking  \
1    SA01      R01     D01 -3.234863 -34.667969 -18.676758  walking   
2    SA01      R01     D01 -5.126953 -37.414551 -16.540527  walking   
3    SA01      R01     D01 -6.347656 -39.489746 -13.854980  walking   
4    SA01      R01     D01 -7.812500 -41.198730 -11.657715  walking   

   scaled_gx  scaled_gy  scaled_gz  
0   0.021823  -0.515393  -0.468850  
1   0.007546  -0.549363  -0.439306  
2  -0.005099  -0.573248  -0.416827  
3  -0.013257  -0.591295  -0.388568  
4  -0.023047  -0.606157  -0.365446  


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical

# DataFrame을 무작위로 섞기
df_shuffled = df.sample(frac=1, random_state=42)

# 데이터를 입력 특성 (X)과 타겟 변수 (y)로 분할하기
X = df_shuffled[['scaled_gx', 'scaled_gy', 'scaled_gz']].values
y = df_shuffled['label'].values

# 레이블을 인코딩하기 위해 LabelEncoder 사용
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# 인코딩된 레이블을 원-핫 인코딩으로 변환하기
y_one_hot = to_categorical(y_encoded)

# 데이터를 훈련 세트와 테스트 세트로 분할하기
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

# 입력 특성을 LSTM 모델의 예상 형태로 재구성
X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_reshaped = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# LSTM 모델 정의
model = Sequential()
model.add(LSTM(64, input_shape=(3, 1), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# LSTM 모델 훈련
model.fit(X_train_reshaped, y_train, epochs=1000, batch_size=32)

# 훈련 데이터에서 모델 평가
train_loss, train_accuracy = model.evaluate(X_train_reshaped, y_train)
print(f'Train Loss: {train_loss:.4f}')
print(f'Train Accuracy: {train_accuracy:.4f}')

# 테스트 데이터에서 모델 평가
test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

Epoch 1/1000
576/576 [==============================] - 4s 2ms/step - loss: 1.4357 - accuracy: 0.3319
Epoch 2/1000
576/576 [==============================] - 1s 2ms/step - loss: 1.4144 - accuracy: 0.3303
Epoch 3/1000
576/576 [==============================] - 1s 2ms/step - loss: 1.3785 - accuracy: 0.3641
Epoch 4/1000
576/576 [==============================] - 1s 2ms/step - loss: 1.3369 - accuracy: 0.3972
Epoch 5/1000
576/576 [==============================] - 1s 2ms/step - loss: 1.3231 - accuracy: 0.4084
Epoch 6/1000
576/576 [==============================] - 1s 2ms/step - loss: 1.3142 - accuracy: 0.4159
Epoch 7/1000
576/576 [==============================] - 1s 2ms/step - loss: 1.3023 - accuracy: 0.4280
Epoch 8/1000
576/576 [==============================] - 1s 2ms/step - loss: 1.2868 - accuracy: 0.4336
Epoch 9/1000
576/576 [==============================] - 1s 2ms/step - loss: 1.2748 - accuracy: 0.4367
Epoch 10/1000
576/576 [==============================] - 1s 2ms/step - loss: 1.273

In [17]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, models

# 데이터 전처리
# 레이블 인코딩
label_encoder = LabelEncoder()
df['label_encoded'] = label_encoder.fit_transform(df['label'])

# 입력 및 출력 데이터 생성
X = df[['scaled_gx', 'scaled_gy', 'scaled_gz']].values
y = df['label_encoded'].values

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 생성
model = models.Sequential()
model.add(layers.Reshape((3, 1), input_shape=(3,)))
model.add(layers.Conv1D(32, 3, activation='relu'))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Conv1D(64, 3, activation='relu'))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 입력 데이터 형태 조정
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

# 모델 훈련
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# 모델 평가
_, train_accuracy = model.evaluate(X_train, y_train)
_, test_accuracy = model.evaluate(X_test, y_test)
print('Train Accuracy:', train_accuracy)
print('Test Accuracy:', test_accuracy)


ValueError: Exception encountered when calling layer "max_pooling1d_3" (type MaxPooling1D).

Negative dimension size caused by subtracting 2 from 1 for '{{node max_pooling1d_3/MaxPool}} = MaxPool[T=DT_FLOAT, data_format="NHWC", explicit_paddings=[], ksize=[1, 2, 1, 1], padding="VALID", strides=[1, 2, 1, 1]](max_pooling1d_3/ExpandDims)' with input shapes: [?,1,1,32].

Call arguments received by layer "max_pooling1d_3" (type MaxPooling1D):
  • inputs=tf.Tensor(shape=(None, 1, 32), dtype=float32)

In [1]:
import os
import pandas as pd

# DataFrame 초기화
df = pd.DataFrame(columns=['user_ID', 'trial_ID', 'task_ID', 'gx', 'gy', 'gz', 'label'])

# task_id와 매핑되는 label 정보 생성
label_mapping = {
    'D01': 'walking',
    'D03': 'jogging',
    'D05': 'stairs_walking',
    'D07': 'standing',
    'F06': 'fall'
}

# 데이터 폴더 경로
data_folder = "../data/SisFall_dataset"

# 폴더 내의 파일들을 확인하고 처리
record_count = 0  # 전체 파일에서의 레코드 개수
for root, dirs, files in os.walk(data_folder):
    for dir_name in dirs:
        dir_path = os.path.join(root, dir_name)
        user_ID = dir_name

        for file_name in os.listdir(dir_path):
            if file_name.endswith(".txt"):
                # 파일 경로
                file_path = os.path.join(dir_path, file_name)

                # 파일 이름에 대한 정보
                activity, _, trial_ID_with_extension = file_name.split('_')
                trial_ID = trial_ID_with_extension[:-4]  # .txt 제거

                # task_id가 D01, D03, D05, D07, F06인 경우에만 처리
                if activity in ['D01', 'D03', 'D05', 'D07', 'F06']:
                    # task_id에 해당하는 label 정보
                    task_ID = activity
                    label = label_mapping[task_ID]

                    # Angular velocity 변환을 위한 공식에 사용되는 값들
                    Range = 2000  # 예시 값을 사용
                    Resolution = 16  # 예시 값을 사용

                    # 파일에서 데이터를 읽어와서 처리
                    with open(file_path, 'r') as file:
                        lines = file.readlines()

                    # 데이터 파싱 및 계산
                    gx_values = []
                    gy_values = []
                    gz_values = []

                    for line in lines:
                        # 쉼표를 기준으로 줄을 분리하여 필드의 값을 가져오기
                        fields = line.strip().split(',')

                        # gx, gy, gz 값 얻기 및 변환 계산
                        gx_value = (2 * Range / (2 ** Resolution)) * float(fields[3])
                        gy_value = (2 * Range / (2 ** Resolution)) * float(fields[4])
                        gz_value = (2 * Range / (2 ** Resolution)) * float(fields[5])

                        gx_values.append(gx_value)
                        gy_values.append(gy_value)
                        gz_values.append(gz_value)

                        if len(gx_values) == 1:
                            # 한 줄의 레코드 처리가 완료되었으므로 DataFrame에 추가
                            df.loc[len(df)] = {
                                'user_ID': user_ID,
                                'trial_ID': trial_ID,
                                'task_ID': task_ID,
                                'gx': gx_values[0],
                                'gy': gy_values[0],
                                'gz': gz_values[0],
                                'label': label
                            }
                            record_count += 1
                            gx_values = []
                            gy_values = []
                            gz_values = []

print(df)

KeyboardInterrupt: 

In [2]:
import os
import pandas as pd

# DataFrame 초기화
df = pd.DataFrame(columns=['user_ID', 'trial_ID', 'task_ID', 'gx', 'gy', 'gz', 'label'])

# task_id와 매핑되는 label 정보 생성
label_mapping = {
    'D01': 'walking',
    'D03': 'jogging',
    'D05': 'stairs_walking',
    'D07': 'standing',
    'F06': 'fall'
}

# 데이터 폴더 경로
data_folder = "../data/SisFall_dataset"

# 폴더 내의 파일들을 확인하고 처리
record_count = 0  # 전체 파일에서의 레코드 개수
for root, dirs, files in os.walk(data_folder):
    for dir_name in dirs:
        dir_path = os.path.join(root, dir_name)
        user_ID = dir_name

        for file_name in os.listdir(dir_path):
            if file_name.endswith(".txt"):
                # 파일 경로
                file_path = os.path.join(dir_path, file_name)

                # 파일 이름에 대한 정보
                activity, _, trial_ID_with_extension = file_name.split('_')
                trial_ID = trial_ID_with_extension[:-4]  # .txt 제거

                # task_id가 D01, D03, D05, D07, F06인 경우에만 처리
                if activity in ['D01', 'D03', 'D05', 'D07', 'F06']:
                    # task_id에 해당하는 label 정보
                    task_ID = activity
                    label = label_mapping[task_ID]

                    # Angular velocity 변환을 위한 공식에 사용되는 값들
                    Range = 2000  # 예시 값을 사용
                    Resolution = 16  # 예시 값을 사용

                    # 파일에서 데이터를 읽어와서 처리
                    with open(file_path, 'r') as file:
                        lines = file.readlines()

                    # 데이터 파싱 및 계산
                    gx_values = []
                    gy_values = []
                    gz_values = []

                    for i, line in enumerate(lines):
                        # 쉼표를 기준으로 줄을 분리하여 필드의 값을 가져오기
                        fields = line.strip().split(',')

                        # gx, gy, gz 값 얻기 및 변환 계산
                        gx_value = (2 * Range / (2 ** Resolution)) * float(fields[3])
                        gy_value = (2 * Range / (2 ** Resolution)) * float(fields[4])
                        gz_value = (2 * Range / (2 ** Resolution)) * float(fields[5])

                        gx_values.append(gx_value)
                        gy_values.append(gy_value)
                        gz_values.append(gz_value)

                        if i % 10 == 0:  # 1, 11, 21, 31번째 행만 처리
                            # 1, 11, 21, 31번째 행의 레코드를 DataFrame에 추가
                            df.loc[len(df)] = {
                                'user_ID': user_ID,
                                'trial_ID': trial_ID,
                                'task_ID': task_ID,
                                'gx': gx_value,
                                'gy': gy_value,
                                'gz': gz_value,
                                'label': label
                            }
                            record_count += 1

print(df)


IndexError: list index out of range

In [ ]:
import os
import pandas as pd

# DataFrame 초기화
df = pd.DataFrame(columns=['user_ID', 'trial_ID', 'task_ID', 'gx', 'gy', 'gz'])

# 데이터 폴더 경로
data_folder = "../data/SisFall_dataset"

# 폴더 내의 파일들을 확인하고 처리
record_count = 0  # 전체 파일에서의 레코드 개수
for root, dirs, files in os.walk(data_folder):
    for dir_name in dirs:
        dir_path = os.path.join(root, dir_name)
        user_ID = dir_name

        for file_name in os.listdir(dir_path):
            if file_name.endswith(".txt"):
                # 파일 경로
                file_path = os.path.join(dir_path, file_name)

                # 파일 이름에 대한 정보
                activity, _, trial_ID_with_extension = file_name.split('_')
                trial_ID = trial_ID_with_extension[:-4]  # .txt 제거

                # task_id가 D01, D03, D05, D07, F06인 경우에만 처리
                if activity in ['D01', 'D03', 'D05', 'D07', 'F06']:
                    # task_id에 해당하는 label 정보
                    task_ID = activity

                    # 파일에서 데이터를 읽어와서 처리
                    with open(file_path, 'r', encoding='utf-8') as file:
                        lines = file.readlines()

                    # 데이터 파싱 및 DataFrame에 추가
                    for i, line in enumerate(lines):
                        if i % 10 == 0:  # 1, 11, 21, 31번째 행만 처리
                            # 쉼표를 기준으로 줄을 분리하여 필드의 값을 가져오기
                            fields = line.strip().split(',')

                            if len(fields) >= 6:  # 필드 수 확인
                                # gx, gy, gz 값 얻기
                                gx = float(fields[3])
                                gy = float(fields[4])
                                gz = float(fields[5])

                                # 1, 11, 21, 31번째 행의 레코드를 DataFrame에 추가
                                df.loc[len(df)] = {
                                    'user_ID': user_ID,
                                    'trial_ID': trial_ID,
                                    'task_ID': task_ID,
                                    'gx': gx,
                                    'gy': gy,
                                    'gz': gz
                                }
                                record_count += 1

print(df)

In [5]:
import os
import pandas as pd

# DataFrame 초기화
df = pd.DataFrame(columns=['user_ID', 'trial_ID', 'task_ID', 'gx', 'gy', 'gz'])

# task_id와 매핑되는 label 정보 생성
label_mapping = {
    'D01': 'walking',
    'D03': 'jogging',
    'D05': 'stairs_walking',
    'D07': 'standing',
    'F06': 'fall'
}

# 데이터 폴더 경로
data_folder = "../data/SisFall_dataset"

# 폴더 내의 파일들을 확인하고 처리
record_count = 0  # 전체 파일에서의 레코드 개수
for root, dirs, files in os.walk(data_folder):
    for dir_name in dirs:
        dir_path = os.path.join(root, dir_name)
        user_ID = dir_name

        for file_name in os.listdir(dir_path):
            if file_name.endswith(".txt"):
                # 파일 경로
                file_path = os.path.join(dir_path, file_name)

                # 파일 이름에 대한 정보
                activity, _, trial_ID_with_extension = file_name.split('_')
                trial_ID = trial_ID_with_extension[:-4]  # .txt 제거

                # task_id가 D01, D03, D05, D07, F06인 경우에만 처리
                if activity in ['D01', 'D03', 'D05', 'D07', 'F06']:
                    # task_id에 해당하는 label 정보
                    task_ID = activity
                    label = label_mapping[task_ID]

                    #Angular velocity 변환을 위한 공식에 사용되는 값들
                    Range = 2000  # 예시 값을 사용
                    Resolution = 16  # 예시 값을 사용

                    # 파일에서 데이터를 읽어와서 처리
                    with open(file_path, 'r', encoding='utf-8') as file:
                        lines = file.readlines()

                    # 데이터 파싱 및 계산
                    gx_values = []
                    gy_values = []
                    gz_values = []

                    # 데이터 파싱 및 DataFrame에 추가
                    for i, line in enumerate(lines):
                        if i % 10 == 0:  # 1, 11, 21, 31번째 행만 처리
                            # 쉼표를 기준으로 줄을 분리하여 필드의 값을 가져오기
                            fields = line.strip().split(',')

                            if len(fields) >= 6:  # 필드 수 확인
                                # gx, gy, gz 값 얻기 및 변환 계산
                                gx_value = (2 * Range / (2 ** Resolution)) * float(fields[3])
                                gy_value = (2 * Range / (2 ** Resolution)) * float(fields[4])
                                gz_value = (2 * Range / (2 ** Resolution)) * float(fields[5])

                                gx_values.append(gx_value)
                                gy_values.append(gy_value)
                                gz_values.append(gz_value)

                                if len(gx_values) == 1:
                                    # 한 줄의 레코드 처리가 완료되었으므로 DataFrame에 추가
                                    df.loc[len(df)] = {
                                        'user_ID': user_ID,
                                        'trial_ID': trial_ID,
                                        'task_ID': task_ID,
                                        'gx': gx_values[0],
                                        'gy': gy_values[0],
                                        'gz': gz_values[0],
                                        'label': label
                                    }
                                    record_count += 1
                                    gx_values = []
                                    gy_values = []
                                    gz_values = []

print(df)


       user_ID trial_ID task_ID         gx         gy         gz
0         SA01      R01     D01  -1.098633 -30.761719 -21.484375
1         SA01      R01     D01 -10.864258 -46.752930  -3.173828
2         SA01      R01     D01  31.860352 -22.216797   8.056641
3         SA01      R01     D01   2.624512 -11.352539  29.052734
4         SA01      R01     D01   7.263184  15.869141  26.184082
...        ...      ...     ...        ...        ...        ...
331797    SE15      R05     D07  -1.281738   3.601074  -0.366211
331798    SE15      R05     D07  -1.586914   3.295898  -0.610352
331799    SE15      R05     D07  -1.586914   3.173828  -0.915527
331800    SE15      R05     D07  -1.525879   2.868652  -0.976562
331801    SE15      R05     D07  -1.281738   3.051758  -0.915527

[331802 rows x 6 columns]


In [6]:
import os
import pandas as pd

# DataFrame 초기화
df = pd.DataFrame(columns=['user_ID', 'trial_ID', 'task_ID', 'gx', 'gy', 'gz', 'label'])

# task_id와 매핑되는 label 정보 생성
label_mapping = {
    'D01': 'walking',
    'D03': 'jogging',
    'D05': 'stairs_walking',
    'D07': 'standing',
    'F06': 'fall'
}

# 데이터 폴더 경로
data_folder = "../data/SisFall_dataset"

# 폴더 내의 파일들을 확인하고 처리
record_count = 0  # 전체 파일에서의 레코드 개수
for root, dirs, files in os.walk(data_folder):
    for dir_name in dirs:
        dir_path = os.path.join(root, dir_name)
        user_ID = dir_name

        for file_name in os.listdir(dir_path):
            if file_name.endswith(".txt"):
                # 파일 경로
                file_path = os.path.join(dir_path, file_name)

                # 파일 이름에 대한 정보
                activity, _, trial_ID_with_extension = file_name.split('_')
                trial_ID = trial_ID_with_extension[:-4]  # .txt 제거

                # task_id가 D01, D03, D05, D07, F06인 경우에만 처리
                if activity in ['D01', 'D03', 'D05', 'D07', 'F06']:
                    # task_id에 해당하는 label 정보
                    task_ID = activity
                    label = label_mapping[task_ID]

                    #Angular velocity 변환을 위한 공식에 사용되는 값들
                    Range = 2000  # 예시 값을 사용
                    Resolution = 16  # 예시 값을 사용

                    # 파일에서 데이터를 읽어와서 처리
                    with open(file_path, 'r', encoding='utf-8') as file:
                        lines = file.readlines()

                    # 데이터 파싱 및 계산
                    gx_values = []
                    gy_values = []
                    gz_values = []

                    # 데이터 파싱 및 DataFrame에 추가
                    for i, line in enumerate(lines):
                        if i % 10 == 0:  # 1, 11, 21, 31번째 행만 처리
                            # 쉼표를 기준으로 줄을 분리하여 필드의 값을 가져오기
                            fields = line.strip().split(',')

                            if len(fields) >= 6:  # 필드 수 확인
                                # gx, gy, gz 값 얻기 및 변환 계산
                                gx_value = (2 * Range / (2 ** Resolution)) * float(fields[3])
                                gy_value = (2 * Range / (2 ** Resolution)) * float(fields[4])
                                gz_value = (2 * Range / (2 ** Resolution)) * float(fields[5])

                                gx_values.append(gx_value)
                                gy_values.append(gy_value)
                                gz_values.append(gz_value)

                                if len(gx_values) == 1:
                                    # 한 줄의 레코드 처리가 완료되었으므로 DataFrame에 추가
                                    df.loc[len(df)] = {
                                        'user_ID': user_ID,
                                        'trial_ID': trial_ID,
                                        'task_ID': task_ID,
                                        'gx': gx_values[0],
                                        'gy': gy_values[0],
                                        'gz': gz_values[0],
                                        'label': label
                                    }
                                    record_count += 1
                                    gx_values = []
                                    gy_values = []
                                    gz_values = []

print(df)


       user_ID trial_ID task_ID         gx         gy         gz     label
0         SA01      R01     D01  -1.098633 -30.761719 -21.484375   walking
1         SA01      R01     D01 -10.864258 -46.752930  -3.173828   walking
2         SA01      R01     D01  31.860352 -22.216797   8.056641   walking
3         SA01      R01     D01   2.624512 -11.352539  29.052734   walking
4         SA01      R01     D01   7.263184  15.869141  26.184082   walking
...        ...      ...     ...        ...        ...        ...       ...
331797    SE15      R05     D07  -1.281738   3.601074  -0.366211  standing
331798    SE15      R05     D07  -1.586914   3.295898  -0.610352  standing
331799    SE15      R05     D07  -1.586914   3.173828  -0.915527  standing
331800    SE15      R05     D07  -1.525879   2.868652  -0.976562  standing
331801    SE15      R05     D07  -1.281738   3.051758  -0.915527  standing

[331802 rows x 7 columns]


In [9]:
# CSV 파일로 저장
csv_path = "../data/SisFall_df_re.csv"
df.to_csv(csv_path, index=False)
print("CSV 파일이 저장되었습니다:", csv_path)

CSV 파일이 저장되었습니다: ../data/SisFall_df_re.csv


In [20]:
from sklearn.preprocessing import MinMaxScaler

# MinMaxScaler 인스턴스 생성
scaler = MinMaxScaler(feature_range=(-1, 1))

# 'gx', 'gy', 'gz' 열의 데이터 추출
data = df[['gx', 'gy', 'gz']].values

# 데이터 스케일링
scaled_data = scaler.fit_transform(data)

# 스케일링된 값을 새로운 열에 저장
df['scaled_gx'] = scaled_data[:, 0]
df['scaled_gy'] = scaled_data[:, 1]
df['scaled_gz'] = scaled_data[:, 2]

print(df.head())

  user_ID trial_ID task_ID         gx         gy         gz    label   
0    SA01      R01     D01  -1.098633 -30.761719 -21.484375  walking  \
1    SA01      R01     D01 -10.864258 -46.752930  -3.173828  walking   
2    SA01      R01     D01  31.860352 -22.216797   8.056641  walking   
3    SA01      R01     D01   2.624512 -11.352539  29.052734  walking   
4    SA01      R01     D01   7.263184  15.869141  26.184082  walking   

   scaled_gx  scaled_gy  scaled_gz  
0   0.015592   0.010843   0.042287  
1   0.001771  -0.010123   0.066825  
2   0.062238   0.022046   0.081875  
3   0.020861   0.036290   0.110011  
4   0.027426   0.071980   0.106167  


In [21]:
# CSV 파일로 저장
csv_path = "../data/SisFall_df_sc.csv"
df.to_csv(csv_path, index=False)
print("CSV 파일이 저장되었습니다:", csv_path)

CSV 파일이 저장되었습니다: ../data/SisFall_df_sc.csv


In [10]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# 데이터셋 준비
X = []  # 입력 데이터
y = []  # 출력 데이터

# 4개의 레코드씩 묶어서 처리
for i in range(0, len(df) - 3, 4):
    # gx, gy, gz 값 가져오기
    gx_values = df.loc[i:i+3, 'gx'].values
    gy_values = df.loc[i:i+3, 'gy'].values
    gz_values = df.loc[i:i+3, 'gz'].values

    # 레이블 값 가져오기
    labels = df.loc[i:i+3, 'label'].values

    # 서로 다른 레이블이 포함된 경우 해당 시퀀스는 분석에서 제외
    if len(set(labels)) == 1:
        # 시퀀스로 변환하여 입력 데이터에 추가
        X.append([gx_values, gy_values, gz_values])

        # 레이블 값 중복 제거하여 출력 데이터에 추가
        y.append(labels[0])

# 입력 데이터와 출력 데이터를 넘파이 배열로 변환
X = np.array(X)
y = np.array(y)

# 출력 데이터를 One-Hot 인코딩
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y = onehot_encoder.fit_transform(integer_encoded)

# LSTM 모델 정의
model = Sequential()
model.add(LSTM(128, input_shape=(3, 4)))  # 3개의 시퀀스, 각 시퀀스에 4개의 피처
model.add(Dense(5, activation='softmax'))  # 분류할 클래스 수에 맞게 조정

# 모델 컴파일 및 학습
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=10, batch_size=32)

D:\Danu\TensorFlow\venv\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Epoch 1/10
2590/2590 [==============================] - 6s 2ms/step - loss: 0.8476 - accuracy: 0.6595
Epoch 2/10
2590/2590 [==============================] - 5s 2ms/step - loss: 0.7481 - accuracy: 0.6978
Epoch 3/10
2590/2590 [==============================] - 5s 2ms/step - loss: 0.7174 - accuracy: 0.7094
Epoch 4/10
2590/2590 [==============================] - 5s 2ms/step - loss: 0.6953 - accuracy: 0.7182
Epoch 5/10
2590/2590 [==============================] - 5s 2ms/step - loss: 0.6763 - accuracy: 0.7250
Epoch 6/10
2590/2590 [==============================] - 5s 2ms/step - loss: 0.6600 - accuracy: 0.7317
Epoch 7/10
2590/2590 [==============================] - 5s 2ms/step - loss: 0.6445 - accuracy: 0.7373
Epoch 8/10
2590/2590 [==============================] - 5s 2ms/step - loss: 0.6307 - accuracy: 0.7426
Epoch 9/10
2590/2590 [==============================] - 5s 2ms/step - loss: 0.6169 - accuracy: 0.7481
Epoch 10/10
2590/2590 [==============================] - 5s 2ms/step - loss: 0.604

In [14]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# 데이터셋 준비
X = []  # 입력 데이터
y = []  # 출력 데이터

# 4개의 레코드씩 묶어서 처리
for i in range(0, len(df) - 3, 4):
    # gx, gy, gz 값 가져오기
    gx_values = df.loc[i:i+3, 'gx'].values
    gy_values = df.loc[i:i+3, 'gy'].values
    gz_values = df.loc[i:i+3, 'gz'].values

    # 레이블 값 가져오기
    labels = df.loc[i:i+3, 'label'].values

    # 서로 다른 레이블이 포함된 경우 해당 시퀀스는 분석에서 제외
    if len(set(labels)) == 1:
        # 시퀀스로 변환하여 입력 데이터에 추가
        X.append([gx_values, gy_values, gz_values])

        # 레이블 값 중복 제거하여 출력 데이터에 추가
        y.append(labels[0])

# 입력 데이터와 출력 데이터를 넘파이 배열로 변환
X = np.array(X)
y = np.array(y)

# 출력 데이터를 One-Hot 인코딩
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y = onehot_encoder.fit_transform(integer_encoded)

# LSTM 모델 정의
model = Sequential()
model.add(LSTM(128, input_shape=(3, 4)))  # 3개의 시퀀스, 각 시퀀스에 4개의 피처
model.add(Dense(5, activation='softmax'))  # 분류할 클래스 수에 맞게 조정

# 모델 컴파일 및 학습
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=10, batch_size=32)

D:\Danu\TensorFlow\venv\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Epoch 1/10
2590/2590 [==============================] - 6s 2ms/step - loss: 0.8456 - accuracy: 0.6605
Epoch 2/10
2590/2590 [==============================] - 5s 2ms/step - loss: 0.7501 - accuracy: 0.6962
Epoch 3/10
2590/2590 [==============================] - 5s 2ms/step - loss: 0.7180 - accuracy: 0.7098
Epoch 4/10
2590/2590 [==============================] - 5s 2ms/step - loss: 0.6968 - accuracy: 0.7163
Epoch 5/10
2590/2590 [==============================] - 5s 2ms/step - loss: 0.6786 - accuracy: 0.7248
Epoch 6/10
2590/2590 [==============================] - 5s 2ms/step - loss: 0.6623 - accuracy: 0.7308
Epoch 7/10
2590/2590 [==============================] - 5s 2ms/step - loss: 0.6464 - accuracy: 0.7364
Epoch 8/10
2590/2590 [==============================] - 5s 2ms/step - loss: 0.6319 - accuracy: 0.7424
Epoch 9/10
2590/2590 [==============================] - 5s 2ms/step - loss: 0.6200 - accuracy: 0.7477
Epoch 10/10
2590/2590 [==============================] - 5s 2ms/step - loss: 0.605

In [15]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# 데이터셋 준비
X = []  # 입력 데이터
y = []  # 출력 데이터

# 4개의 레코드씩 묶어서 처리
for i in range(0, len(df) - 3, 4):
    # gx, gy, gz 값 가져오기
    gx_values = df.loc[i:i+3, 'gx'].values
    gy_values = df.loc[i:i+3, 'gy'].values
    gz_values = df.loc[i:i+3, 'gz'].values

    # 레이블 값 가져오기
    labels = df.loc[i:i+3, 'label'].values

    # 서로 다른 레이블이 포함된 경우 해당 시퀀스는 분석에서 제외
    if len(set(labels)) == 1:
        # 시퀀스로 변환하여 입력 데이터에 추가
        X.append([gx_values, gy_values, gz_values])

        # 레이블 값 중복 제거하여 출력 데이터에 추가
        y.append(labels[0])

# 입력 데이터와 출력 데이터를 넘파이 배열로 변환
X = np.array(X)
y = np.array(y)

# 출력 데이터를 One-Hot 인코딩
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y = onehot_encoder.fit_transform(integer_encoded)

# LSTM 모델 정의
model = Sequential()
model.add(LSTM(128, input_shape=(3, 4)))  # 3개의 시퀀스, 각 시퀀스에 4개의 피처
model.add(Dense(5, activation='softmax'))  # 분류할 클래스 수에 맞게 조정

# 모델 컴파일 및 학습
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, batch_size=32)

D:\Danu\TensorFlow\venv\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Epoch 1/100
2590/2590 [==============================] - 6s 2ms/step - loss: 0.8447 - accuracy: 0.6600
Epoch 2/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.7475 - accuracy: 0.6986
Epoch 3/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.7184 - accuracy: 0.7079
Epoch 4/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.6969 - accuracy: 0.7169
Epoch 5/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.6784 - accuracy: 0.7246
Epoch 6/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.6601 - accuracy: 0.7323
Epoch 7/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.6467 - accuracy: 0.7368
Epoch 8/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.6318 - accuracy: 0.7430
Epoch 9/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.6183 - accuracy: 0.7484
Epoch 10/100
2590/2590 [==============================] - 5s 2ms/step - l

In [22]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

# 데이터셋 준비
X = []  # 입력 데이터
y = []  # 출력 데이터

# 4개의 레코드씩 묶어서 처리
for i in range(0, len(df) - 3, 4):
    # gx, gy, gz 값 가져오기
    gx_values = df.loc[i:i+3, 'scaled_gx'].values
    gy_values = df.loc[i:i+3, 'scaled_gy'].values
    gz_values = df.loc[i:i+3, 'scaled_gz'].values

    # 레이블 값 가져오기
    labels = df.loc[i:i+3, 'label'].values

    # 서로 다른 레이블이 포함된 경우 해당 시퀀스는 분석에서 제외
    if len(set(labels)) == 1:
        # 시퀀스로 변환하여 입력 데이터에 추가
        X.append([gx_values, gy_values, gz_values])

        # 레이블 값 중복 제거하여 출력 데이터에 추가
        y.append(labels[0])

# 입력 데이터와 출력 데이터를 넘파이 배열로 변환
X = np.array(X)
y = np.array(y)

# 출력 데이터를 One-Hot 인코딩
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y)
onehot_encoder = OneHotEncoder(sparse_output=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y = onehot_encoder.fit_transform(integer_encoded)

# 데이터를 학습용과 테스트용으로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 설정된 시드를 사용하여 TensorFlow의 재현성 확보
tf.random.set_seed(42)

# LSTM 모델 정의
model = Sequential()
model.add(LSTM(128, input_shape=(3, 4)))  # 3개의 시퀀스, 각 시퀀스에 4개의 피처
model.add(Dense(5, activation='softmax'))  # 분류할 클래스 수에 맞게 조정

# 모델 컴파일 및 학습
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/100
2072/2072 [==============================] - 7s 2ms/step - loss: 1.5363 - accuracy: 0.3052 - val_loss: 1.3873 - val_accuracy: 0.3980
Epoch 2/100
2072/2072 [==============================] - 5s 2ms/step - loss: 1.2636 - accuracy: 0.4559 - val_loss: 1.2174 - val_accuracy: 0.4922
Epoch 3/100
2072/2072 [==============================] - 5s 2ms/step - loss: 1.1875 - accuracy: 0.4858 - val_loss: 1.1634 - val_accuracy: 0.5111
Epoch 4/100
2072/2072 [==============================] - 6s 3ms/step - loss: 1.1385 - accuracy: 0.5104 - val_loss: 1.0999 - val_accuracy: 0.5325
Epoch 5/100
2072/2072 [==============================] - 5s 2ms/step - loss: 1.0690 - accuracy: 0.5444 - val_loss: 1.0422 - val_accuracy: 0.5645
Epoch 6/100
2072/2072 [==============================] - 5s 2ms/step - loss: 1.0224 - accuracy: 0.5676 - val_loss: 1.0390 - val_accuracy: 0.5488
Epoch 7/100
2072/2072 [==============================] - 5s 2ms/step - loss: 0.9948 - accuracy: 0.5790 - val_loss: 0.9900 - val_ac

In [1]:
import pandas as pd

df = pd.read_csv("../data/SisFall_df_sc.csv")

print(df)

       user_ID trial_ID task_ID         gx         gy         gz     label   
0         SA01      R01     D01  -1.098633 -30.761719 -21.484375   walking  \
1         SA01      R01     D01 -10.864258 -46.752930  -3.173828   walking   
2         SA01      R01     D01  31.860352 -22.216797   8.056641   walking   
3         SA01      R01     D01   2.624512 -11.352539  29.052734   walking   
4         SA01      R01     D01   7.263184  15.869141  26.184082   walking   
...        ...      ...     ...        ...        ...        ...       ...   
331797    SE15      R05     D07  -1.281738   3.601074  -0.366211  standing   
331798    SE15      R05     D07  -1.586914   3.295898  -0.610352  standing   
331799    SE15      R05     D07  -1.586914   3.173828  -0.915527  standing   
331800    SE15      R05     D07  -1.525879   2.868652  -0.976562  standing   
331801    SE15      R05     D07  -1.281738   3.051758  -0.915527  standing   

        scaled_gx  scaled_gy  scaled_gz  
0        0.015592   0

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
import pickle

# 데이터셋 준비
X = []  # 입력 데이터
y = []  # 출력 데이터

# 4개의 레코드씩 묶어서 처리
for i in range(0, len(df) - 3, 4):
    # gx, gy, gz 값 가져오기
    gx_values = df.loc[i:i+3, 'scaled_gx'].values
    gy_values = df.loc[i:i+3, 'scaled_gy'].values
    gz_values = df.loc[i:i+3, 'scaled_gz'].values

    # 레이블 값 가져오기
    labels = df.loc[i:i+3, 'label'].values

    # 서로 다른 레이블이 포함된 경우 해당 시퀀스는 분석에서 제외
    if len(set(labels)) == 1:
        # 시퀀스로 변환하여 입력 데이터에 추가
        X.append([gx_values, gy_values, gz_values])

        # 레이블 값 중복 제거하여 출력 데이터에 추가
        y.append(labels[0])

# 입력 데이터와 출력 데이터를 넘파이 배열로 변환
X = np.array(X)
y = np.array(y)

# 출력 데이터를 One-Hot 인코딩
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y)
onehot_encoder = OneHotEncoder(sparse_output=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y = onehot_encoder.fit_transform(integer_encoded)

# 데이터를 학습용과 테스트용으로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 설정된 시드를 사용하여 TensorFlow의 재현성 확보
tf.random.set_seed(42)

# LSTM 모델 정의
model = Sequential()
model.add(LSTM(128, input_shape=(3, 4)))  # 3개의 시퀀스, 각 시퀀스에 4개의 피처
model.add(Dense(5, activation='softmax'))  # 분류할 클래스 수에 맞게 조정

# 모델 컴파일 및 학습
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=1000, batch_size=64, validation_data=(X_test, y_test))

# 최상의 적합 모델을 피클 파일로 저장
best_model = history.model
pickle.dump(best_model, open('best_model.pkl', 'wb'))

Epoch 1/1000
1036/1036 [==============================] - 5s 3ms/step - loss: 1.5518 - accuracy: 0.2892 - val_loss: 1.5486 - val_accuracy: 0.2877
Epoch 2/1000
1036/1036 [==============================] - 3s 3ms/step - loss: 1.3602 - accuracy: 0.4109 - val_loss: 1.2887 - val_accuracy: 0.4293
Epoch 3/1000
1036/1036 [==============================] - 3s 3ms/step - loss: 1.2494 - accuracy: 0.4556 - val_loss: 1.2233 - val_accuracy: 0.4604
Epoch 4/1000
1036/1036 [==============================] - 3s 3ms/step - loss: 1.1873 - accuracy: 0.4862 - val_loss: 1.1833 - val_accuracy: 0.4967
Epoch 5/1000
1036/1036 [==============================] - 3s 3ms/step - loss: 1.1649 - accuracy: 0.4979 - val_loss: 1.1632 - val_accuracy: 0.5056
Epoch 6/1000
1036/1036 [==============================] - 3s 3ms/step - loss: 1.1531 - accuracy: 0.5030 - val_loss: 1.1589 - val_accuracy: 0.4871
Epoch 7/1000
1036/1036 [==============================] - 3s 3ms/step - loss: 1.1198 - accuracy: 0.5168 - val_loss: 1.0919 -

KeyboardInterrupt: 

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
import pickle

# 데이터 불러오기
df = pd.read_csv('../data/SisFall_df_sc.csv')

# 데이터셋 준비
X = []  # 입력 데이터
y = []  # 출력 데이터

# 4개의 레코드씩 묶어서 처리
for i in range(0, len(df) - 3, 4):
    # gx, gy, gz 값 가져오기
    gx_values = df.loc[i:i+3, 'scaled_gx'].values
    gy_values = df.loc[i:i+3, 'scaled_gy'].values
    gz_values = df.loc[i:i+3, 'scaled_gz'].values

    # 레이블 값 가져오기
    labels = df.loc[i:i+3, 'label'].values

    # 서로 다른 레이블이 포함된 경우 해당 시퀀스는 분석에서 제외
    if len(set(labels)) == 1:
        # 시퀀스로 변환하여 입력 데이터에 추가
        X.append([gx_values, gy_values, gz_values])

        # 레이블 값 중복 제거하여 출력 데이터에 추가
        y.append(labels[0])

# 입력 데이터와 출력 데이터를 넘파이 배열로 변환
X = np.array(X)
y = np.array(y)

# 출력 데이터를 One-Hot 인코딩
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y)
onehot_encoder = OneHotEncoder(sparse_output=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y = onehot_encoder.fit_transform(integer_encoded)

# 시퀀스 단위로 데이터를 분할하기 위한 인덱스 계산
seq_length = 4
split_index = range(seq_length, len(X), seq_length)

# 시퀀스 단위로 데이터 분할
X_seq = []
y_seq = []
for i in split_index:
    X_seq.append(X[i-seq_length:i])
    y_seq.append(y[i-seq_length:i])

# 분할된 시퀀스 데이터를 넘파이 배열로 변환
X_seq = np.array(X_seq)
y_seq = np.array(y_seq)

# 데이터를 학습용과 테스트용으로 분할
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

# 설정된 시드를 사용하여 TensorFlow의 재현성 확보
tf.random.set_seed(42)

# LSTM 모델 정의
model = Sequential()
model.add(LSTM(128, input_shape=(seq_length, 3)))  # seq_length개의 시퀀스, 각 시퀀스에 3개의 피처
model.add(Dense(5, activation='softmax'))  # 분류할 클래스 수에 맞게 조정

# 모델 컴파일 및 학습
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_test, y_test))

# 최상의 적합 모델을 피클 파일로 저장
best_model = history.model
pickle.dump(best_model, open('best_model.pkl', 'wb'))


Epoch 1/100


ValueError: in user code:

    File "C:\Users\SW402-08\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\SW402-08\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\SW402-08\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\SW402-08\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\SW402-08\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File 

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
import pickle

# 데이터 불러오기
df = pd.read_csv('../data/SisFall_df_sc.csv')

# 데이터셋 준비
X = []  # 입력 데이터
y = []  # 출력 데이터

# 4개의 레코드씩 묶어서 처리
for i in range(0, len(df) - 3, 4):
    # gx, gy, gz 값 가져오기
    gx_values = df.loc[i:i+3, 'scaled_gx'].values
    gy_values = df.loc[i:i+3, 'scaled_gy'].values
    gz_values = df.loc[i:i+3, 'scaled_gz'].values

    # 레이블 값 가져오기
    labels = df.loc[i:i+3, 'label'].values

    # 서로 다른 레이블이 포함된 경우 해당 시퀀스는 분석에서 제외
    if len(set(labels)) == 1:
        # 시퀀스로 변환하여 입력 데이터에 추가
        X.append([gx_values, gy_values, gz_values])

        # 레이블 값 중복 제거하여 출력 데이터에 추가
        y.append(labels[0])

# 입력 데이터와 출력 데이터를 넘파이 배열로 변환
X = np.array(X)
y = np.array(y)

# 출력 데이터를 One-Hot 인코딩
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y = onehot_encoder.fit_transform(integer_encoded)

# 시퀀스 단위로 데이터를 분할하기 위한 인덱스 계산
seq_length = 4
split_index = range(seq_length, len(X), seq_length)

# 시퀀스 단위로 데이터 분할
X_seq = []
y_seq = []
for i in split_index:
    X_seq.append(X[i-seq_length:i])
    y_seq.append(y[i-seq_length:i])

# 분할된 시퀀스 데이터를 넘파이 배열로 변환
X_seq = np.array(X_seq)
y_seq = np.array(y_seq)

# 데이터를 학습용과 테스트용으로 분할
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

# 설정된 시드를 사용하여 TensorFlow의 재현성 확보
tf.random.set_seed(42)

# LSTM 모델 정의
model = Sequential()
model.add(LSTM(128, input_shape=(seq_length, 3)))  # 3개의 시퀀스, 각 시퀀스에 3개의 피처
model.add(Dense(5, activation='softmax'))  # 분류할 클래스 수에 맞게 조정

# 모델 컴파일 및 학습
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_test, y_test))

# 최상의 적합 모델을 피클 파일로 저장
best_model = history.model
pickle.dump(best_model, open('best_model.pkl', 'wb'))

D:\Danu\TensorFlow\venv\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Epoch 1/100


ValueError: in user code:

    File "C:\Users\SW402-08\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\SW402-08\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\SW402-08\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\SW402-08\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\SW402-08\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File 

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# 데이터셋 준비
X = []  # 입력 데이터
y = []  # 출력 데이터

# 4개의 레코드씩 묶어서 처리
for i in range(0, len(df) - 3, 4):
    # gx, gy, gz 값 가져오기
    gx_values = df.loc[i:i+3, 'gx'].values
    gy_values = df.loc[i:i+3, 'gy'].values
    gz_values = df.loc[i:i+3, 'gz'].values

    # 레이블 값 가져오기
    labels = df.loc[i:i+3, 'label'].values

    # 서로 다른 레이블이 포함된 경우 해당 시퀀스는 분석에서 제외
    if len(set(labels)) == 1:
        # 시퀀스로 변환하여 입력 데이터에 추가
        X.append([gx_values, gy_values, gz_values])

        # 레이블 값 중복 제거하여 출력 데이터에 추가
        y.append(labels[0])

# 입력 데이터와 출력 데이터를 넘파이 배열로 변환
X = np.array(X)
y = np.array(y)

# 출력 데이터를 One-Hot 인코딩
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y = onehot_encoder.fit_transform(integer_encoded)

# LSTM 모델 정의
model = Sequential()
model.add(LSTM(128, input_shape=(3, 4)))  # 3개의 시퀀스, 각 시퀀스에 4개의 피처
model.add(Dense(5, activation='softmax'))  # 분류할 클래스 수에 맞게 조정

# 모델 컴파일 및 학습
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, batch_size=32)

In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
import pickle

# 데이터셋 준비
X = []  # 입력 데이터
y = []  # 출력 데이터

# 4개의 레코드씩 묶어서 처리
for i in range(0, len(df) - 3, 4):
    # gx, gy, gz 값 가져오기
    gx_values = df.loc[i:i+3, 'gx'].values
    gy_values = df.loc[i:i+3, 'gy'].values
    gz_values = df.loc[i:i+3, 'gz'].values

    # 레이블 값 가져오기
    labels = df.loc[i:i+3, 'label'].values

    # 서로 다른 레이블이 포함된 경우 해당 시퀀스는 분석에서 제외
    if len(set(labels)) == 1:
        # 시퀀스로 변환하여 입력 데이터에 추가
        X.append([gx_values, gy_values, gz_values])

        # 레이블 값 중복 제거하여 출력 데이터에 추가
        y.append(labels[0])

# 입력 데이터와 출력 데이터를 넘파이 배열로 변환
X = np.array(X)
y = np.array(y)

# 출력 데이터를 One-Hot 인코딩
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y = onehot_encoder.fit_transform(integer_encoded)

# LSTM 모델 정의
model = Sequential()
model.add(LSTM(128, input_shape=(3, 4)))  # 3개의 시퀀스, 각 시퀀스에 4개의 피처
model.add(Dense(5, activation='softmax'))  # 분류할 클래스 수에 맞게 조정

# 모델 컴파일 및 학습
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X, y, epochs=100, batch_size=32)

# 최상의 적합 모델을 피클 파일로 저장
best_model = history.model
pickle.dump(best_model, open('../model/best_model.pkl', 'wb'))

D:\Danu\TensorFlow\venv\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Epoch 1/100
2590/2590 [==============================] - 7s 2ms/step - loss: 0.8483 - accuracy: 0.6587
Epoch 2/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.7500 - accuracy: 0.6974
Epoch 3/100
2590/2590 [==============================] - 6s 2ms/step - loss: 0.7188 - accuracy: 0.7101
Epoch 4/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.6958 - accuracy: 0.7168
Epoch 5/100
2590/2590 [==============================] - 6s 2ms/step - loss: 0.6778 - accuracy: 0.7251
Epoch 6/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.6616 - accuracy: 0.7293
Epoch 7/100
2590/2590 [==============================] - 6s 2ms/step - loss: 0.6459 - accuracy: 0.7367
Epoch 8/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.6302 - accuracy: 0.7439
Epoch 9/100
2590/2590 [==============================] - 5s 2ms/step - loss: 0.6174 - accuracy: 0.7491
Epoch 10/100
2590/2590 [==============================] - 5s 2ms/step - l

INFO:tensorflow:Assets written to: ram://d960d710-78d4-4e80-9b1c-f1d46209e812/assets


INFO:tensorflow:Assets written to: ram://d960d710-78d4-4e80-9b1c-f1d46209e812/assets


In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# 예측할 데이터 불러오기
pred_df = pd.read_csv('../data/Arduino/fall_sc.csv')

# 입력 데이터 준비
X_pred = []  # 입력 데이터

# 4개의 레코드씩 묶어서 처리
for i in range(0, len(pred_df) - 3, 4):
    # gx, gy, gz 값 가져오기
    gx_values = pred_df.loc[i:i+3, 'scaled_gx'].values
    gy_values = pred_df.loc[i:i+3, 'scaled_gy'].values
    gz_values = pred_df.loc[i:i+3, 'scaled_gz'].values

    # 시퀀스로 변환하여 입력 데이터에 추가
    X_pred.append([gx_values, gy_values, gz_values])

# 입력 데이터를 넘파이 배열로 변환
X_pred = np.array(X_pred)

# 모델을 사용하여 예측
predictions = best_model.predict(X_pred)

# 예측 결과 디코딩
label_encoder = LabelEncoder()
label_encoder.classes_ = np.load('label_encoder_classes.npy')
decoded_predictions = label_encoder.inverse_transform(np.argmax(predictions, axis=1))

# 예측 결과를 CSV 파일로 저장
pred_df['predicted_label'] = decoded_predictions
pred_df.to_csv('../data/prediction_results.csv', index=False)

2/2 [==============================] - 0s 0s/step


FileNotFoundError: [Errno 2] No such file or directory: 'label_encoder_classes.npy'